In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from imblearn.combine import SMOTEENN
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
import re
from collections import Counter
import pickle


In [10]:
df = pd.read_csv('Topics.csv')

In [11]:
df.head()

,word,topic
0,ha,Casual Conversations
1,ni,Casual Conversations
2,kr,Casual Conversations
3,message,Casual Conversations
4,ma,Casual Conversations


In [12]:
df['topic'].value_counts()

Casual Conversations         2676
Academic Discussions          216
Everyday Queries              152
Informal Discussions          109
School or Group Chat           80
General Updates                34
Religious References           18
Casual Banter                  17
Political Discussions          15
Relationship Talks             13
General News and Updates       11
Health and Wellness             8
Shopping Discussions            5
Sports Discussions              5
Travel Discussions              5
Music Conversations             4
Entertainment Discussions       4
Family Matters                  4
Academic Performance            4
Food and Cooking                3
Business and Economy            1
Imran Khan                      1
Name: topic, dtype: int64

In [13]:
# Filter out rows where 'topics' is either 'Imran Khan' or 'Business and Economy'
df = df[~df['topic'].isin(['Imran Khan', 'Business and Economy'])]


In [14]:
df['topic'].value_counts()

Casual Conversations         2676
Academic Discussions          216
Everyday Queries              152
Informal Discussions          109
School or Group Chat           80
General Updates                34
Religious References           18
Casual Banter                  17
Political Discussions          15
Relationship Talks             13
General News and Updates       11
Health and Wellness             8
Shopping Discussions            5
Sports Discussions              5
Travel Discussions              5
Music Conversations             4
Family Matters                  4
Academic Performance            4
Entertainment Discussions       4
Food and Cooking                3
Name: topic, dtype: int64

In [15]:
df.describe

<bound method NDFrame.describe of          word                 topic
0          ha  Casual Conversations
1          ni  Casual Conversations
2          kr  Casual Conversations
3     message  Casual Conversations
4          ma  Casual Conversations
...       ...                   ...
3380    javed  Casual Conversations
3381    maina  Casual Conversations
3382   bhaijo  Casual Conversations
3383   sunain  Casual Conversations
3384     lift    Travel Discussions

[3383 rows x 2 columns]>

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3383 entries, 0 to 3384
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    3382 non-null   object
 1   topic   3383 non-null   object
dtypes: object(2)
memory usage: 79.3+ KB


In [17]:
df.isnull().sum()

word     1
topic    0
dtype: int64

In [18]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
3380    False
3381    False
3382    False
3383    False
3384    False
Length: 3383, dtype: bool

In [19]:
df['word'] = df['word'].fillna('')

In [20]:
# Step 1: Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['word'])

smote = SMOTE(k_neighbors=2)

# Step 3: Apply SMOTEENN with the SMOTE instance
smote_enn = SMOTEENN(smote=smote)
X_resampled, y_resampled = smote_enn.fit_resample(X, df['topic'])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.2, random_state=42)

In [22]:
model = RandomForestClassifier(class_weight='balanced')
model.fit(X_train, y_train)



RandomForestClassifier(class_weight='balanced')

In [23]:
# Predict
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report

# Evaluate
print(classification_report(y_test, y_pred))

                           precision    recall  f1-score   support

     Academic Discussions       1.00      0.93      0.96       541
     Academic Performance       1.00      1.00      1.00       547
            Casual Banter       1.00      1.00      1.00       570
     Casual Conversations       0.82      1.00      0.90       527
Entertainment Discussions       1.00      1.00      1.00       529
         Everyday Queries       1.00      0.94      0.97       542
           Family Matters       1.00      1.00      1.00       548
         Food and Cooking       1.00      1.00      1.00       521
 General News and Updates       1.00      1.00      1.00       526
          General Updates       1.00      0.99      0.99       548
      Health and Wellness       1.00      1.00      1.00       519
     Informal Discussions       1.00      0.97      0.98       547
      Music Conversations       1.00      1.00      1.00       587
    Political Discussions       1.00      1.00      1.00     

In [24]:
f = open('i.txt','r',encoding='utf-8')

In [25]:
data = f.read()

In [26]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s?[ap]m\s-\s'

In [27]:
messages = re.split(pattern,data)[1:]
messages

['Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n',
 'Raffay created group "TUF (IT) Election"\n',
 "You joined using this group's invite link\n",
 'Raffay: Polling has been started cast your vote ASAP...2nd floor\n',
 'Hamza Malik Tuf A: *Anyone who needs assistance for get out of Class just for the sake of Voting Contact Me*\n',
 'Zayn Jutt💸🤙🏻: Thanks everyone for their support it’s really means alot❤️\n',
 'Zayn Jutt💸🤙🏻: Alhadulillah we won🤲🏻❤️\n',
 '+92 304 1012203: Ma Sha Allah ♥️♥️♥️\n',
 '+92 301 7103186: MashaAllah ♥️👍🏻\n',
 "Zayn Jutt💸🤙🏻 changed this group's settings to allow all members to send messages to this group\n",
 'Zayn Jutt💸🤙🏻: Bht shukriya sb bhaioun kay pyar kaa❤️\n',
 '+92 304 0020606: 101 votes, MashAllah\n',
 'Raffay: Alhamdullilah 😌❤️\n',
 'Mian mahad: MashAllah our both candidates win with huge margin , Alhamdulillah ❤️\n',
 '+92 306 1646447: Congratulations buddy ❤️❤

In [28]:
dates = re.findall(pattern,data)

dates = [item.replace('\u202f', ' ') for item in dates]

dates = [item.replace(' - ', '') for item in dates]



In [29]:
df = pd.DataFrame({'user_message': messages, 'message_date': dates})

In [30]:
users = []
messages = []

for message in df['user_message']:
     entry = re.split('([\w\W]+?):\s',message)
     if entry[1:]:
         users.append(entry[1])
         messages.append(entry[2])
     else:
         users.append('group_notification')
         messages.append(entry[0])

df['user'] =users
df['message'] = messages

df.drop(columns=['user_message'],inplace=True)

      

In [31]:
df = df[df['user'] != 'group_notification' ]
df = df[ df['user'] != 'Meta AI']
df = df[  df['message'] != '<Media omitted>\n']


In [32]:
def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove links
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove emojis (This is a basic regex for emojis, might not cover all cases)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = text.strip()  # Optional: to remove leading and trailing whitespace
    return text


# Apply the clean_text function to the 'message' column
df['cleaned_message'] = df['message'].apply(clean_text)



In [33]:
new_data = df['cleaned_message']

# Step 1: Vectorize the new input data
new_vectorized = vectorizer.transform(new_data)

# Step 2: Make predictions
predictions = model.predict(new_vectorized)


In [34]:
# Step 3: Output the predictions and count occurrences
topic_counts = Counter()
new_data = df['cleaned_message']
for text, prediction in zip(new_data, predictions):
    topic_counts[prediction] += 1  # Increment the count for the predicted topic

# Step 4: Display the most used topics with counts
print("\nMost Used Topics with Counts:")
for topic, count in topic_counts.items():
    print(f"{topic}: {count}")

print("\nMost Used Topics with Counts:")
for topic, count in topic_counts.most_common(4):  # Get the top 3 most common topics
    print(f"{topic}: {count}")


Most Used Topics with Counts:
Political Discussions: 2
School or Group Chat: 2
Casual Conversations: 43
Sports Discussions: 2
Informal Discussions: 1
Relationship Talks: 3
Religious References: 1

Most Used Topics with Counts:
Casual Conversations: 43
Relationship Talks: 3
Political Discussions: 2
School or Group Chat: 2


In [35]:
import pickle
pickle.dump(model, open("model.pkl", "wb"))

pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))

In [36]:
import sklearn
print(sklearn.__version__)


1.0.2


In [37]:
import sys
import pkg_resources

# Check Python executable path
print("Python executable:", sys.executable)

# List installed packages
installed_packages = pkg_resources.working_set
installed_packages_list = sorted(["{}=={}".format(i.key, i.version) for i in installed_packages])
print("\nInstalled packages:")
print("\n".join(installed_packages_list))



Python executable: C:\Users\abuba\anaconda3\envs\myenv\python.exe

Installed packages:
anyio==4.2.0
argon2-cffi-bindings==21.2.0
argon2-cffi==21.3.0
asttokens==2.0.5
async-lru==2.0.4
attrs==23.1.0
autocommand==2.2.2
babel==2.11.0
backcall==0.2.0
backports.tarfile==1.2.0
beautifulsoup4==4.12.3
bleach==4.1.0
brotli==1.0.9
certifi==2024.7.4
cffi==1.16.0
charset-normalizer==3.3.2
colorama==0.4.6
comm==0.2.1
debugpy==1.6.7
decorator==5.1.1
defusedxml==0.7.1
exceptiongroup==1.2.0
executing==0.8.3
fastjsonschema==2.16.2
idna==3.7
imbalanced-learn==0.12.3
importlib-metadata==7.0.1
importlib-resources==6.4.0
inflect==7.3.1
ipykernel==6.28.0
ipython==8.12.2
ipywidgets==8.1.2
jaraco.context==5.3.0
jaraco.functools==4.0.1
jaraco.text==3.12.1
jedi==0.19.1
jinja2==3.1.4
joblib==1.4.2
json5==0.9.6
jsonschema-specifications==2023.7.1
jsonschema==4.19.2
jupyter-client==8.6.0
jupyter-console==6.6.3
jupyter-core==5.7.2
jupyter-events==0.10.0
jupyter-lsp==2.2.0
jupyter-server-terminals==0.4.4
jupyter-serv

In [38]:
import scipy
print(scipy.__version__)


1.7.3


In [39]:
import pandas
print(pandas.__version__)

1.1.5


In [45]:
import numpy 
print(numpy.__version__)

1.21.6


In [49]:
print("Numpy version:", numpy.__version__)
print("Scipy version:", scipy.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
# print("Matplotlib version:", matplotlib.__version__)
# print("Seaborn version:", seaborn.__version__)

Numpy version: 1.21.6
Scipy version: 1.7.3
Scikit-learn version: 1.0.2
Pandas version: 1.1.5
